## 依赖包
<!-- **Author**: `zwy` -->


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

import random
import shutil

plt.ion()   # 交互模式可以动态显示图像

### 硬件选择 GPU

In [ ]:
# GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## 数据加载及处理


### 下载数据集
可以从Google Drive / Github / 其他数据地址下载数据

In [ ]:
!rm -rf CNN
!git clone https://github.com/Zwysun/CNN.git
!ls

Cloning into 'CNN'...
remote: Enumerating objects: 705, done.
remote: Counting objects: 100% (705/705), done.
remote: Compressing objects: 100% (562/562), done.
remote: Total 705 (delta 163), reused 680 (delta 141), pack-reused 0
Receiving objects: 100% (705/705), 31.29 MiB | 15.97 MiB/s, done.
Resolving deltas: 100% (163/163), done.
CNN  sample_data


### 数据预处理
利用transforms对图片进行预处理，可以分别针对训练集和验证集采取不同的处理方法


In [ ]:
# 直接加载数据集MINST

# 训练集 扩充及正则化
# 测试集 仅正则化

# 图像变换操作
data_transforms = {
    'train': transforms.Compose([
        # transforms.RandomResizedCrop(224),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.1307], [0.3081])
    ]),
    'test': transforms.Compose([
        # transforms.Resize(256),
        # transforms.CenterCrop(224),
        transforms.ToTensor(),
        # transforms.Normalize([0.5], [0.5])
        transforms.Normalize([0.1307], [0.3081])
    ]),
}

# 下载公共数据集

train_dataset = datasets.MNIST( root="./mnist",  # 设置数据集的根目录
    train=True,  # 是否是训练集
    transform=data_transforms['train'],  # 对数据进行转换
    download=True
    )
test_dataset = datasets.MNIST( root="./mnist", 
    train=False,  # 测试集，所以false
    transform=data_transforms['test'], 
    download=True
    )


Extracting ./mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist/MNIST/raw


Extracting ./mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist/MNIST/raw



Extracting ./mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist/MNIST/raw


Extracting ./mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
image_datasets = {'train':train_dataset, 'test':test_dataset}
# 数据
dataloaders = {x:torch.utils.data.DataLoader(image_datasets[x], 
                batch_size=64, shuffle=True, num_workers=4)
                for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['test'].classes
print (dataset_sizes)
print (image_datasets)
print (class_names)

{'train': 60000, 'test': 10000}
{'train': Dataset MNIST
    Number of datapoints: 60000
    Root location: ./mnist
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0.1307], std=[0.3081])
           ), 'test': Dataset MNIST
    Number of datapoints: 10000
    Root location: ./mnist
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0.1307], std=[0.3081])
           )}
['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']


### 显示部分图片

In [ ]:
# def imshow(inp, title=None):
#     """Imshow for Tensor."""
#     inp = inp.numpy().transpose((1, 2, 0))
#     # mean = np.array([0.485, 0.456, 0.406])
#     # std = np.array([0.229, 0.224, 0.225])   # RGB
#     mean = np.array([0.1307])
#     std = np.array([0.3081])   # 灰度图
    
#     inp = std * inp + mean
#     inp = np.clip(inp, 0, 1)
#     plt.imshow(inp)
#     if title is not None:
#         plt.title(title)
#     plt.pause(0.001)  # pause a bit so that plots are updated

# # Get a batch of training data
# inputs, classes = next(iter(dataloaders['val']))

# # Make a grid from batch
# out = torchvision.utils.make_grid(inputs)

# imshow(out, title=[class_names[x] for x in classes])

## 模型训练

In [ ]:
#训练过程
def train_model(model, dataloaders, loss_function, optimizer, num_epochs=25):
  since = time.time()
  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0
  train_loader = dataloaders['train']
  val_loader = dataloaders['test']

  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-' * 10)
    # 记录把所有数据集训练+测试一遍需要多长时间
    startTick = time.time()

    model.train()
    running_loss = 0.0
    running_corrects = 0
    for inputs, labels in train_loader:  # 对于训练集的每一个batch
      inputs = inputs.to(device)
      labels = labels.to(device)
      
      out = model(inputs)  # 送进网络进行输出
      _, preds = torch.max(out, 1)
      loss = loss_function(out, labels)  # 获得损失
  
      optimizer.zero_grad()  # 梯度归零
      loss.backward()  # 反向传播获得梯度，但是参数还没有更新
      optimizer.step()  # 更新梯度
      # scheduler.step()  # 更新学习率

      # statistics
      running_loss += loss.item() * inputs.size(0)
      running_corrects += torch.sum(preds == labels.data)
    
    epoch_loss = running_loss / dataset_sizes['train']
    epoch_acc = running_corrects.double() / dataset_sizes['train']
    print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                'train', epoch_loss, epoch_acc))

    model.eval()
    running_loss = 0.0
    running_corrects = 0
    for inputs, labels in val_loader:
      inputs = inputs.to(device)
      labels = labels.to(device)
  
      out = model(inputs)  # 获得输出
  
      _, preds = torch.max(out, 1)
      loss = loss_function(out, labels)  # 获得损失
      # torch.max()返回两个结果，
      # 第一个是最大值，第二个是对应的索引值；
      # 第二个参数 0 代表按列取最大值并返回对应的行索引值，1 代表按行取最大值并返回对应的列索引值。
      # statistics
      running_loss += loss.item() * inputs.size(0)
      running_corrects += torch.sum(preds == labels.data)  # 找出预测和真实值相同的数量，也就是以预测正确的数量
    
    epoch_loss = running_loss / dataset_sizes['test']
    epoch_acc = running_corrects.double() / dataset_sizes['test']
    print('{} Loss: {:.4f} Acc: {:.4f}'.format('val', epoch_loss, epoch_acc))
                
    timeSpan = time.time() - startTick
    print("Time: %dS"%(timeSpan))
    print()

    if epoch_acc > best_acc:
      best_acc = epoch_acc
      best_model_wts = copy.deepcopy(model.state_dict())

  time_elapsed = time.time() - since
  print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
  print('Best val Acc: {:4f}'.format(best_acc))
  # load best model weights
  model.load_state_dict(best_model_wts)
  return model

## 搭建网络结构

In [ ]:
# Some example

input(28, 28, 1)
conv(28, 28, 16)
relu(28, 28, 16)
pool(14, 14, 16)
conv(14, 14, 32)
relu(14, 14, 32)
pool(7, 7, 32)
fullyconn(1, 1, 10)
softmax(1, 1, 10)

In [ ]:
# 搭建网络
class myNet(nn.Module):
  def __init__(self):
    super(myNet, self).__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(          #(1,28,28)
        in_channels=1,
        out_channels=16,
        kernel_size=5,
        stride=1,
        padding=2   #padding=(kernelsize-stride)/2
        ),#(16,28,28)
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)#(16,14,14)
        )
    self.conv2 = nn.Sequential(#(16,14,14)
        nn.Conv2d(16, 32, 5, 1, 2),#(32,14,14)
        nn.ReLU(),#(32,14,14)
        nn.MaxPool2d(2),#(32,7,7)
        )
    self.out = torch.nn.Sequential(
        torch.nn.Linear(32*7*7,1024),
        torch.nn.ReLU(),
        # torch.nn.Dropout(p=0.5),
        torch.nn.Linear(1024, 10))
    # self.out = torch.nn.Linear(32*7*7,10)
  def forward(self,x):
    x = self.conv1(x)
    x = self.conv2(x) #(batch,32,7,7)
    x = x.view(x.size(0), -1) #(batch,32*7*7)
    output = self.out(x)
    return output

# class myNet(torch.nn.Module):
#   def __init__(self):
#       super(myNet, self).__init__()
#       self.conv1 = torch.nn.Sequential(
#           torch.nn.Conv2d(1,64,kernel_size=3,stride=1,padding=1),
#           torch.nn.ReLU(),
#           torch.nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
#           torch.nn.ReLU(),
#           torch.nn.MaxPool2d(stride=2,kernel_size=2))
#       self.dense = torch.nn.Sequential(
#           torch.nn.Linear(14*14*128,1024),
#           torch.nn.ReLU(),
#           torch.nn.Dropout(p=0.5),
#           torch.nn.Linear(1024, 10))
#   def forward(self, x):
#       x = self.conv1(x)
#       x = x.view(-1, 14*14*128)
#       x = self.dense(x)
#       return x

# 
model_ft=myNet()
model_ft.to(device)
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
# optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = optim.Adam(model_ft.parameters())

# Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

print(model_ft)


myNet(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=25088, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=1024, out_features=10, bias=True)
  )
)


In [ ]:
# 搭配下面工具，轻松实现可视化
import torch.onnx
from torch.autograd import Variable

x = Variable(torch.randn(1, 1, 28, 28)).to(device)
torch_out = torch.onnx.export(model_ft, x, "test.onnx", export_params=True, verbose=True)
# https://lutzroeder.github.io/netron/

graph(%input.1 : Float(1:784, 1:784, 28:28, 28:1),
      %conv1.0.weight : Float(64:9, 1:9, 3:3, 3:1),
      %conv1.0.bias : Float(64:1),
      %conv1.2.weight : Float(128:576, 64:9, 3:3, 3:1),
      %conv1.2.bias : Float(128:1),
      %dense.0.weight : Float(1024:25088, 25088:1),
      %dense.0.bias : Float(1024:1),
      %dense.3.weight : Float(10:1024, 1024:1),
      %dense.3.bias : Float(10:1)):
  %9 : Float(1:50176, 64:784, 28:28, 28:1) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[3, 3], pads=[1, 1, 1, 1], strides=[1, 1]](%input.1, %conv1.0.weight, %conv1.0.bias) # /usr/local/lib/python3.6/dist-packages/torch/nn/modules/conv.py:416:0
  %10 : Float(1:50176, 64:784, 28:28, 28:1) = onnx::Relu(%9) # /usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1119:0
  %11 : Float(1:100352, 128:784, 28:28, 28:1) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[3, 3], pads=[1, 1, 1, 1], strides=[1, 1]](%10, %conv1.2.weight, %conv1.2.bias) # /usr/local/lib/python3.6/dist-p

## 训练

In [ ]:
model_ft = train_model(model_ft, dataloaders, criterion, optimizer_ft, num_epochs=5)


Epoch 1/5
----------
train Loss: 0.1192 Acc: 0.9627
val Loss: 0.0376 Acc: 0.9880
Time: 11S

Epoch 2/5
----------
train Loss: 0.0383 Acc: 0.9881
val Loss: 0.0352 Acc: 0.9881
Time: 11S

Epoch 3/5
----------
train Loss: 0.0278 Acc: 0.9914
val Loss: 0.0312 Acc: 0.9903
Time: 11S

Epoch 4/5
----------
train Loss: 0.0216 Acc: 0.9932
val Loss: 0.0332 Acc: 0.9897
Time: 11S

Epoch 5/5
----------
train Loss: 0.0159 Acc: 0.9946
val Loss: 0.0307 Acc: 0.9892
Time: 11S

Training complete in 0m 58s
Best val Acc: 0.990300


### 保存

In [ ]:
# 可视化工具
# https://lutzroeder.github.io/netron/
os.chdir("/content")
torch.save(model_ft,'draw.pth')

In [ ]:
# os.chdir("/content/drive/My Drive")
# !ls
# torch.save(model_ft,'light.pth')